<a href="https://colab.research.google.com/github/coleterrell97/portfolio/blob/master/ML_Practice/gpu_based_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [381]:
import torch
import torchvision
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from google.colab.patches import cv2_imshow
EPOCHS = 10

In [382]:
dataset = MNIST(root=".", transform=ToTensor(), download=True)

In [383]:
val_size = 10000
train_size = 60000-val_size
train_set, val_set = random_split(dataset, [train_size, val_size])

train_dl = DataLoader(train_set, batch_size = 4096, shuffle=True)
val_dl = DataLoader(val_set, batch_size = 1024, shuffle=True)

In [384]:
class Net(nn.Module):
  def __init__(self, inputs, hidden_layer, outputs):
    super(Net, self).__init__()
    self.input_layer = nn.Linear(inputs, hidden_layer)
    self.hidden_layer1 = nn.Linear(hidden_layer, 50)
    self.hidden_layer2 = nn.Linear(50, outputs)
  
  def forward(self, inputs):
    inputs = inputs.view(inputs.size(0), -1)
    x = F.relu(self.input_layer(inputs))
    x = F.relu(self.hidden_layer1(x))
    x = self.hidden_layer2(x)
    return x

  def fit(self, dl, epochs, loss_fn, optimizer):
    for epoch in range(epochs):
      loss = None
      for xb, labels in dl:
        if xb == None:
          continue
        predictions = model(xb)
        loss = loss_fn(predictions, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
      print(loss)

In [385]:
model = Net(784, 200, 10)
loss_fn = F.cross_entropy
optimizer = torch.optim.Adam(model.parameters())

In [386]:
def get_default_device():
  if torch.cuda.is_available():
    return torch.device("cuda")
  else:
    return torch.device("cpu")

In [387]:
device = get_default_device()

In [388]:
def to_device(data, device):
  if isinstance(data, (list,tuple)):
      return [to_device(x, device) for x in data]
  return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [389]:
train_loader = DeviceDataLoader(train_dl, device)
val_loader = DeviceDataLoader(val_dl, device)

In [ ]:
to_device(model, device)
model.fit(train_loader, EPOCHS, loss_fn, optimizer)